# IBM Forecasts

**Author:** Andrew Thut
    
This notebook collects data from the featurizer product in IBM's weather API's to produce a geojson file that can be visualised through software like ESRI or [geojson.io](https://geojson.io/)

Please direct inquiries to Andrew Thut: asthut@gmail.com

**Background**

The Weather Company (formally WSI), an IBM Business, helps people make informed decisions and take action in the face of weather. The company offers the most personalized, and actionable weather data and insights to millions of consumers, as well as thousands of marketers and businesses via Weather’s API, its business solutions division, and its own digital products from [The Weather Channel](https://weather.com/) and [Weather Underground](https://www.wunderground.com/).

According to a recent [study](https://newsroom.ibm.com/2021-07-29-IBMs-The-Weather-Company-Continues-to-Be-the-Worlds-Most-Accurate-Forecaster-Overall) by ForecastWatch, The Weather Company was determined to be the most accurate global weather forecaster. The data also revealed The gap between The Weather Company and the next best overall provider has incresed every year of the study.

**Import necessary modules**

In [1]:
import requests,json,datetime
from dotenv import load_dotenv
load_dotenv() #take environment variables from .env
import os

**Authenticate access to API**
To make my API key secret, the code below will retrieve an API key from a file on my local machine.

If you didn't want to hide the key, the code could simply be specified as follows. This approach also would not require the dotenv and os modules.

APIKEY='YourAPIkey'

In [2]:
APIKEY=os.getenv('APIKEY')

**Assign Product Name and Product Number***

This information as well as additional context on how to construct an API call can be found in the documentation linked below. 

[Current Conditions Imagery](https://www.ibm.com/docs/en/environmental-intel-suite?topic=apis-current-conditions-forecast-imagery-layers) <br>
[Forecast Conditions Imagery](https://www.ibm.com/docs/en/environmental-intel-suite?topic=apis-current-conditions-forecast-imagery-layers) <br>
[HailVision](https://www.ibm.com/docs/en/environmental-intel-suite?topic=apis-radar-derived-hailvision): Radar derived hax hail size during previous time period<br>
[HailZone](https://www.ibm.com/docs/en/environmental-intel-suite?topic=apis-radar-derived-hailzone): 1km resolution hail forecast out to 30 minutes<br>
[IceVision](https://www.ibm.com/docs/en/environmental-intel-suite?topic=apis-radar-derived-icevision): Radar derived ice accumulation during previous time period<br>
[Lightning StrikeZone](https://www.ibm.com/docs/en/environmental-intel-suite?topic=apis-radar-derived-lightning-strikezone): 1km resolution lightning forecast out to 30 minutes<br>
[RainVision](https://www.ibm.com/docs/en/environmental-intel-suite?topic=apis-radar-derived-rainvision): Radar derived rain accumulation during previous time period <br>
[ShearVision](https://www.ibm.com/docs/en/environmental-intel-suite?topic=apis-radar-derived-shearvision): Radar derived rotational shear <br>
[SnowVision](https://www.ibm.com/docs/en/environmental-intel-suite?topic=apis-radar-derived-snowvision): Radar derived rain accumulation during previous time period <br>

In [3]:
########################PRODUCT INVENTORY CALL################################  
def APIcall(calltime):
    ##All TWC visual products have a product name and number
    prod_num=str(407) # Product number
    prod_name='MESHTrack1440min' # Per TWC naming conventions
    
    fc = {
        "type": "FeatureCollection",
        "features": []
    }
    
#Alternate Bounding Box for North America
#    bbox='-140.0,25.0,-50.0,50.0'
    
####  Now make threshold calls for each month

    low_limit=[1.0,12.7,25.4,38.1]
    high_limit=[12.7,25.4,38.1,50.8]
    num_layers = len(low_limit)
    
    INVurl = 'https://api.weather.com/v2/tiler/info?products='+prod_num+':'+prod_name+'&apiKey='+APIKEY
    print("Inventory Call: " + INVurl)
    INVAPIcall = requests.get(INVurl) #Our API call to grab the latest forecast
    INVdata = INVAPIcall.json()
    for x in range(0,len(INVdata["layers"][prod_num][prod_name]["dimensions"])):
        t = INVdata["layers"][prod_num][prod_name]["dimensions"][x]["t"][0]
#convert unix time to real time
        timestamp=int(t)/1000
        now= datetime.datetime.utcfromtimestamp(timestamp)
        year=str(now.year)
        month=str('{:02d}'.format(now.month))
        day=str('{:02d}'.format(now.day))
        hour=str('{:02d}'.format(now.hour))
        min=str('{:02d}'.format(now.minute))
        ts=year+"-"+month+"-"+day+","+hour+min
        
########################PRODUCT FEATURE CALL################################
        base1 = 'https://api.weather.com/v2/featurizer/isoband?product='
        base2 = prod_num+':'+prod_name+'&t=' + t
        base3 = '&apiKey='+APIKEY+'&geometryType=polygon'

        for y in range(0,num_layers):
            low_limitS=str(low_limit[y])
            high_limitS=str(high_limit[y])
            base4='&threshold='+low_limitS+':'+high_limitS  #+'&bbox='+bbox
            url = base1 + base2 + base3 + base4  # make API URL
            print ("Feature Call: " + url)
            data = requests.get(url).json() #Our API Call

            for z in data['features']:
                f = {"type": "Feature", "properties": {}, "geometry": None}
                f['geometry'] = z['geometry']
                f['properties']['HailVision'] = z['properties']['threshold']
                f['properties']['Date'] = ts
                fc['features'].append(f)

    filename="HailVision"+calltime+".geojson"
    with open(filename, "w") as outfile:
        json.dump(fc, outfile)

################################################################################
# Get Time
now=datetime.datetime.utcnow()
year=str(now.year)
month=str('{:02d}'.format(now.month))
day=str('{:02d}'.format(now.day))
hour=str('{:02d}'.format(now.hour))
minute=str('{:02d}'.format(now.minute))
calltime=year+month+day+hour+minute

APIcall(calltime) # Function to grab data

/var/folders/lb/j47gsvnn0vng4hq35pt3j0rw0000gn/T/ipykernel_20834/1679380515.py:63: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now=datetime.datetime.utcnow()


Inventory Call: https://api.weather.com/v2/tiler/info?products=407:MESHTrack1440min&apiKey=adb91efe0a544845b91efe0a542845f3


/var/folders/lb/j47gsvnn0vng4hq35pt3j0rw0000gn/T/ipykernel_20834/1679380515.py:29: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  now= datetime.datetime.utcfromtimestamp(timestamp)


Feature Call: https://api.weather.com/v2/featurizer/isoband?product=407:MESHTrack1440min&t=1766093400000&apiKey=adb91efe0a544845b91efe0a542845f3&geometryType=polygon&threshold=1.0:12.7
Feature Call: https://api.weather.com/v2/featurizer/isoband?product=407:MESHTrack1440min&t=1766093400000&apiKey=adb91efe0a544845b91efe0a542845f3&geometryType=polygon&threshold=12.7:25.4
Feature Call: https://api.weather.com/v2/featurizer/isoband?product=407:MESHTrack1440min&t=1766093400000&apiKey=adb91efe0a544845b91efe0a542845f3&geometryType=polygon&threshold=25.4:38.1
Feature Call: https://api.weather.com/v2/featurizer/isoband?product=407:MESHTrack1440min&t=1766093400000&apiKey=adb91efe0a544845b91efe0a542845f3&geometryType=polygon&threshold=38.1:50.8
Feature Call: https://api.weather.com/v2/featurizer/isoband?product=407:MESHTrack1440min&t=1766091600000&apiKey=adb91efe0a544845b91efe0a542845f3&geometryType=polygon&threshold=1.0:12.7
Feature Call: https://api.weather.com/v2/featurizer/isoband?product=407: